<a href="https://colab.research.google.com/github/aso1901102/AI_tech2020/blob/master/2020AI0402_Morphological.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##形態素解析（Morphological Analysis)

### 形態素解析ライプラリMecabinstall

In [1]:
# 形態素解析ライブラリMecabと辞書（meCab-ipadic-NEologd)のインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!pip install mecab-python3 > /dev/null

# MeCab実行時の指定パスをインストールパスにリンク
# シンボリックリンク(/etc/mecabrcを/usr/local/etc/mecabrcで参照できるようにする)
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

ln: failed to create symbolic link '/usr/local/etc/mecabrc': File exists


### Mecabのデフォルト辞書で文章解析

In [2]:
# インストールしたMecabをimport
import MeCab

#MeCabオブジェクトの生成（デフォルト辞書）
tagger = MeCab.Tagger()

# 形態素解析を実行
result = tagger.parse("メイが恋ダンスを踊っている")

# 解析結果を表示
print(result)

メイ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋	名詞,一般,*,*,*,*,恋,コイ,コイ
ダンス	名詞,サ変接続,*,*,*,*,ダンス,ダンス,ダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS



### 最新の情報が入った辞書(ipadic-neologd)で形態素解析

In [3]:
# gitからipadic-neologdの最新版をDL
! git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
# ipadic-neologdのinstall
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n /dev/null 2>&1


fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

### 辞書のインストール済みディレクトリを確認

In [4]:
# 辞書のインストール済みディレクトリの場所を表示
!echo `mecab-config --dicdir`"mecab-ipadic-neologd"


/usr/lib/x86_64-linux-gnu/mecab/dicmecab-ipadic-neologd


### 辞書の場所(ipadic-meologd)の場所を指定してMecabで形態素解析

In [9]:
import MeCab

# mecab-ipadic-NEologd辞書を指定してオブジェクトの生成
path_neologd = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
tagger = MeCab.Tagger("-d" + path_neologd)

# オブジェクトインスタンスで形態素解析
result = tagger.parse("メイが恋ダンスを踊っている")
print(result)

メイ	名詞,固有名詞,人名,一般,*,*,M.A.Y,メイ,メイ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
恋ダンス	名詞,固有名詞,一般,*,*,*,恋ダンス,コイダンス,コイダンス
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
踊っ	動詞,自立,*,*,五段・ラ行,連用タ接続,踊る,オドッ,オドッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS



### 形態素解析毛一家を品詞分析して名詞、動詞、形容詞だけを抜き出してストップワードを除外

In [15]:
import MeCab

# mecab-ipadic-NEologd辞書を指定してオブジェクトの生成
path_neologd = "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
tagger = MeCab.Tagger("-d" + path_neologd)

# 形態素解析して、結果を取得
node = tagger.parseToNode("メイが恋ダンスを踊っている")
print(node)

# 抽出結果を登録する配列
result = []

# リストのリンクNextがNoneになるまでループ
while node is not None:
  # 品詞情報取得　node詳細の先頭(品詞情報)だけを抜き出す
  hinshi = node.feature.split(",")[0]
  if hinshi in ["名詞"]:
    #名詞だったら、表層系を結果配列resultに追加
    result.append(node.surface)
  elif hinshi in ["動詞","形容詞"]:
    result.append(node.feature.split(",")[6])
  # if　ここまで
  node = node.next
# while　ここまで
print(result) 

<Swig Object of type 'MeCab::Node *' at 0x7f2688753d18>
['メイ', '恋ダンス', '踊る', 'いる']
